In [0]:
!pip install transformers
!pip install seqeval
!pip install tensorboardx
!pip install torch==1.4+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!rm -rf .git
!rm -rf apex
!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
!pip install simpletransformers
!pip install Afinn

Looking in links: https://download.pytorch.org/whl/cu100/torch_stable.html
Cloning into 'apex'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 5780 (delta 63), reused 49 (delta 32), pack-reused 5679
Receiving objects: 100% (5780/5780), 13.58 MiB | 30.42 MiB/s, done.
Resolving deltas: 100% (3737/3737), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-b281__5x
Created temporary directory: /tmp/pip-req-tracker-efgbewhq
Created requirements tracker '/tmp/pip-req-tracker-efgbewhq'
Created temporary directory: /tmp/pip-install-q4tpmhwk
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-0iu68lfl
  Added file:///content/ap

In [0]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /content/drive/My\ Drive/NewImplementation

/content/drive/My Drive/NewImplementation


In [0]:
#import subprocess
#subprocess.call(['./installpackages.sh'])

In [0]:

import numpy as np
import pandas as pd
import torch
import copy


from CsvReader import ReadCsv
from HelperFunctions import ShuffleData, replace_all, ReplaceCategoriesWithIndex, ReplaceCategoriesWithIndexOneHot, OneHotEncodingForCategories, TransformDataIntoDataframe, getMetricsMulti
from TrainEvalModel import TrainModelForMultiLabel, EvalFromModel
from NonMLMethods import LogisticRegressionMulti

from google.colab import drive
drive.mount("/content/drive")
data, categories, tendencies = ReadCsv('/content/drive/My Drive/Colab Notebooks/etiki_data','test.csv','companies.csv', 'categories.csv','references.csv','tendencies.csv', 'topics.csv')

for i in range(2,6):
  #categoryData = ReplaceCategoriesWithIndex(categories, data[:,[0,4,13]],True)
  #This is the new better code
  categoryData = data[:,[0,4,13]]
  baselineData = ReplaceCategoriesWithIndex(categories, data[:,[0,4,13]],False)
  
  baselineData = ShuffleData(baselineData)
  #categoryData = ShuffleData(OneHotEncodingForCategories(categoryData))
  
   
  
  trainDataBL = np.array(baselineData[:int(len(baselineData)*0.7)]) 
  print(trainDataBL)
  testDataBL = baselineData[int(len(baselineData)*0.7):] 

  train_dfBL = TransformDataIntoDataframe(trainDataBL)
  eval_dfBL = TransformDataIntoDataframe(testDataBL)

  newArray = []
  for entry in testDataBL:
    newEntry = [entry[0],list(entry[1]), entry[2]]
    newArray.append(newEntry)

  cm_testDataBL = OneHotEncodingForCategories(np.array(newArray))
  cm_testData_dfBL = TransformDataIntoDataframe(cm_testDataBL)
  

  #trainData = categoryData[:int(len(categoryData)*0.7)] 
  #testData = categoryData[int(len(categoryData)*0.7):]
  #This is the new better code
  trainData = np.array(OneHotEncodingForCategories(ReplaceCategoriesWithIndexOneHot(categories,np.array(categoryData[:int(len(categoryData)*0.7)]))))
  testData = OneHotEncodingForCategories(ReplaceCategoriesWithIndex(categories,baselineData[int(len(baselineData)*0.7):],True))

  train_df = TransformDataIntoDataframe(trainData)
  eval_df = TransformDataIntoDataframe(testData)
  
  pred = LogisticRegressionMulti(train_dfBL, eval_dfBL)
  

  algo = 'xlnet'
  args = {'reprocess_input_data': True,
            'overwrite_output_dir': True,
            'num_train_epochs': i,
            'silent':True,
            'max_seq_length': 128
            }

  model = TrainModelForMultiLabel(algo, 'xlnet-base-cased',train_df,5,args)


  import sklearn
  from sklearn.metrics import multilabel_confusion_matrix
  from sklearn.metrics import classification_report
  # Evaluate the model
  result, model_outputs, wrong_predictions = model.eval_model(eval_df)
  #prediction, raw_outputs = model.predict(eval_df)
  #print(prediction)

  threshold = 0.5
  op = []
  for output in model_outputs:
    li = output
    res = [1 if el > threshold else 0 for el in li]
    op.append(res)

  eval_labels = np.array(eval_df["label"].tolist())
  out = np.array(op)
  cm = multilabel_confusion_matrix(eval_labels, out)
  print ("XLNET")
  getMetricsMulti(cm,algo, str(i)+' epoch_sep')

  algo = 'bert'
  args = {'reprocess_input_data': True,
            'overwrite_output_dir': True,
            'num_train_epochs': i,
            'silent':True,
            'max_seq_length': 128
            }

  model = TrainModelForMultiLabel('bert', 'bert-base-cased',train_df,5,args)

  import sklearn
  from sklearn.metrics import multilabel_confusion_matrix
  from sklearn.metrics import classification_report
  # Evaluate the model
  result, model_outputs, wrong_predictions = model.eval_model(eval_df)
  
  threshold = 0.5
  op = []
  for output in model_outputs:
    li = output
    res = [1 if el > threshold else 0 for el in li]
    op.append(res)

  eval_labels = np.array(eval_df["label"].tolist())
  out = np.array(op)
  cm = multilabel_confusion_matrix(eval_labels, out)
  print ("BERT")
  getMetricsMulti(cm,algo, str(i)+' epoch_sep')

  algo = 'LogisticRegression'
  prediction = LogisticRegressionMulti(train_dfBL, eval_dfBL)
  

  threshold = 0.5
  op = []
  for output in prediction:
    li = output
    res = [1 if el > threshold else 0 for el in li]
    op.append(res)

  eval_labels = np.array(cm_testData_dfBL["label"].tolist())
  out = np.array(op)
  cm = multilabel_confusion_matrix(eval_labels, out)
  print ("LOGISTIC REGRESSION")
  getMetricsMulti(cm,algo, str(i)+' epoch_sep')

  #import csv
  #with open('results/'+algo+'-out.csv', 'a', newline='') as f:
    #writer = csv.writer(f)
    #writer.writerows(out)

  #with open('results/'+algo+'-eval.csv', 'a', newline='') as f:
    #writer = csv.writer(f)
    #writer.writerows(eval_labels)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[['35' '4'
  '"Every Parent Should Know The Scandalous History Of Infant FormulaBaby Milk Action pdfNew York City\'s campaign against infant formula inspired us to look into the dubious history of this product.Outrage started in the 1970s, when Nestle was accused of getting third world mothers hooked on formula, which is less healthy and more expensive than breast milk.The allegations led to hearings in the Senate and the World Health Organization, resulting in a new set of marketing rules.Yet infant formula remains a $11.5-billion-and-growing market.\'The Baby Killer\' blew the lid off the formula industry in 1974.Social rights groups began dragging the industry\'s exploitative practices into the spotlight in the early 1970s.The New Internationalist published an exposé on Nestlé\'s marketing practices in 1973, ""Babies Mean Business,"" which described how th

Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:244: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


...

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:574: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


BERT
Confusion Matrix:
[[[75 25]
  [20  4]]

 [[96  9]
  [17  2]]

 [[74 23]
  [24  3]]

 [[69 23]
  [25  7]]

 [[94  8]
  [22  0]]]
Precision: 0.16666666666666666
Recall: 0.13793103448275862
F1-Score: 0.1509433962264151
Precision: 0.10526315789473684
Recall: 0.18181818181818182
F1-Score: 0.13333333333333333
Precision: 0.1111111111111111
Recall: 0.11538461538461539
F1-Score: 0.11320754716981132
Precision: 0.21875
Recall: 0.23333333333333334
F1-Score: 0.22580645161290322
Precision: 0.0
Recall: 0.0
F1-Score: nan


/content/drive/My Drive/NewImplementation/HelperFunctions.py:136: RuntimeWarning: invalid value encountered in double_scalars
  foneScore = 2 * ((precision*recall)/(precision+recall))


LOGISTIC REGRESSION
Confusion Matrix:
[[[118   0]
  [ 30   2]]

 [[128   0]
  [ 21   1]]

 [[120   0]
  [ 30   0]]

 [[108   2]
  [ 18  22]]

 [[122   2]
  [ 26   0]]]
Precision: 0.0625
Recall: 1.0
F1-Score: 0.11764705882352941
Precision: 0.045454545454545456
Recall: 1.0
F1-Score: 0.08695652173913045
Precision: 0.0
Recall: nan
F1-Score: nan
Precision: 0.55
Recall: 0.9166666666666666
F1-Score: 0.6874999999999999
Precision: 0.0
Recall: 0.0
F1-Score: nan


/content/drive/My Drive/NewImplementation/HelperFunctions.py:135: RuntimeWarning: invalid value encountered in long_scalars
  recall = TP/(TP+FN)
/content/drive/My Drive/NewImplementation/HelperFunctions.py:136: RuntimeWarning: invalid value encountered in double_scalars
  foneScore = 2 * ((precision*recall)/(precision+recall))
